In [1]:
import time
import pandas as pd
import numpy as np
import math

In [2]:
daily_demands = {'1': [(42.1,7.6),(41.9,12),(40.2,12.3),(37.2,12.3),(36.1,12.3),(34.6,12.6),(34.3,12.7)],
                '2': [(15.6,4.8),(15.3,7.9),(15.8,7.7),(14.8,8.1),(16.2,7.7),(15.9,7.8),(17.2,7.7)],
                '3': [(30,8.2),(29.3,12.3),(27.2,12.9),(26,14),(25.9,14.1),(25.6,14.4),(26.1,14.3)],
                '4': [(3.1,1.6),(2.8,2.5),(3,2.5),(2.5,2.6),(2.8,2.6),(2.7,2.6),(3.2,2.5)],
                '5': [(7.9,3.1),(8.4,4.8),(7.9,4.8),(7.3,5),(7.4,5),(6.9,5),(8.1,5)],
                '6': [(22.8,5.8),(21.4,9),(19.9,9.1),(16.4,10.1),(20.4,10.2),(20.1,10.4),(16.6,10.4)], 
                '7': [(3.1,1.5),(2.9,2.5),(3.6,2.5),(3.1,2.4),(5,2.4),(5,2.5),(4.1,2.4)], 
                '8': [(3.8,2.4),(4.2,3.8),(4,3.7),(4,4.1),(3.4,4),(3.4,3.9),(3.4,3.8)], 
                '9': [(7.9,3.9),(8.7,5.6),(9,5.6),(9.1,6),(9.4,6),(9.4,6),(9.3,5.9)]} 

In [3]:
store_grades = {'1': 3, '2': 1, '3': 1, '4': 3, '5': 2, '6': 2, '7': 4, '8': 3, '9': 4}

In [4]:
grade_counts = {}
for grade in store_grades.values():
    if grade in grade_counts:
        grade_counts[grade] += 1
    else:
        grade_counts[grade] = 1

grade_counts

{3: 3, 1: 2, 2: 2, 4: 2}

In [5]:
mean_sum = {
    k: round(sum(val[0] for val in v), 2)
    for k, v in daily_demands.items()
}

In [6]:
current_inventory={}
for store,demand  in daily_demands.items():
    total = int(sum(x[0] for x in demand))
    current_inventory[store]=total

In [7]:
current_inventory = {'1': 81,
                    '2': 38,
                    '3': 25,
                    '4': 200,
                    '5': 43,
                    '6': 90,
                    '7': 162,
                    '8': 20,
                    '9': 200}

In [8]:
initial_inventory = current_inventory.copy()

In [9]:
def reset_inventory():
    return initial_inventory.copy()

In [10]:
num_stores = 9
current_day = 4

In [11]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [12]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [13]:
sales_dff = pd.DataFrame(columns=['Store', 'Day', 'Sales','Lost Sales','Demand','Inventory'])

In [14]:
def update_inventory(i,current_day):
    global sales_dff
    todays_sales = {i: max(0,int(np.random.normal(daily_demands[(i)][current_day][0], daily_demands[(i)][current_day][1])))
                    }
    actual_sales = min(todays_sales[i], current_inventory[i])
    lost_sales = max(0, todays_sales[i] - current_inventory[i])
    current_inventory[i] = current_inventory[i]-actual_sales
    new_row = pd.DataFrame({'Store': [i], 'Day': [current_day], 'Sales': [actual_sales],'Lost Sales':lost_sales,'Demand':todays_sales[i],'Inventory':current_inventory[i]})
    sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)
    return current_inventory

In [15]:
def reset_inventory():
    return initial_inventory.copy()

In [16]:
# updated
def shipper_candidates(current_day):
    shipper_cand = []
   
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = int(mean) + 1
        
        if critical_point < update_inventory(str(i),current_day)[i]:
            shipper_cand.append(i)
    return shipper_cand

In [17]:
# updated
def receiving_candidates(current_day):
    receiving_cand = []
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = int(mean) - 1
      
        if critical_point > current_inventory[i]:
            receiving_cand.append(i)
    return receiving_cand

In [18]:
current_time=time.time()
week = 1
results = pd.DataFrame()
for i in range(2000):
    
    current_inventory = reset_inventory()
    
    #print(f"{week}. beginning of the week, inventory: {current_inventory}")
    weekly_df = pd.DataFrame(columns=["week", "day", "shipper", "receiver", "transshipment_amount"])
    sales_dff = pd.DataFrame()
    current_day = 0
    for i in range(6):
        
        max_transshipments = {}
        max_needs = {}
        receiver_inventory = {}
        receiver_demand = {}
        receiver_st_dev = {}
        if current_day <= 5:
            
            shipping_list = shipper_candidates(current_day)
            receiving_list = receiving_candidates(current_day)
            #print(f"shipping list: {shipping_list}, receiving list: {receiving_list}")
            daily_df = pd.DataFrame(columns=["shipper", "receiver", "transshipment_amount", "transshipment_cost"])
           
            for i in shipping_list:
                s_inv = current_inventory[i]
                s_mean, s_stdev = convolution(i,current_day)            
                
                max_transshipments[i] = max(0, (int(s_inv) - int(s_mean)))
                
                for k in receiving_list:
                    r_mean, r_stdev = convolution(k,current_day)
                    r_inv = current_inventory[k]
                    
                    max_needs[k] = max(0, (int(r_mean) - int(r_inv)))
                    receiver_inventory[k] = r_inv
                    receiver_demand[k] = r_mean
                    receiver_st_dev[k] = r_stdev
                    
            max_n_amount = sum(max_needs.values())
            max_t_amount = sum(max_transshipments.values())
            
            if max_n_amount > max_t_amount:
                #print(f"receiver_inventory: {receiver_inventory}")
                #print(f"receiver_demands: {receiver_demand}")
                sum_r_inv = sum(receiver_inventory.values())
                sum_r_demand = sum(receiver_demand.values())
                sum_r_st_dev = sum(receiver_st_dev.values())
                payda = sum_r_demand + (sum_r_st_dev*(1.8339))
                transshipment_amounts = {}
                
                for k in receiving_list:
                    r_mean, r_stdev = convolution(k,current_day)
                    r_inv = current_inventory[k]
                    #print(f"receiver, receiver_mean, receiver_inventory: {k}, {r_mean}, {r_inv}")
                    occured_transhipment = max(0, int(((r_mean + (r_stdev*1.75))/payda)*max_t_amount))
                    #print(f"occured transshipment= {occured_transhipment}")
                    transshipment_amounts[k] = occured_transhipment
                
                transshipment_amounts = {k: v for k, v in transshipment_amounts.items() if v != 0}
                #print(f"transshipment_amounts: {transshipment_amounts}")
                
                df = pd.DataFrame.from_dict(current_inventory, orient='index', columns=['inventory'])
                df['max_transship'] = df.index.map(max_transshipments).fillna(0).astype(int)
                df['need'] = df.index.map(transshipment_amounts).fillna(0).astype(int)
                transfers = []
                while df['max_transship'].sum() > 0 and df['need'].sum() > 0:
                    # sender: max_transship > 0 ve en yüksek
                    sender_row = df[df['max_transship'] > 0].sort_values(by='max_transship', ascending=False).iloc[0]
                    sender_id = sender_row.name
                    sender_capacity = sender_row['max_transship']
                
                    # receiver: need > 0 ve en yüksek
                    receiver_row = df[df['need'] > 0].sort_values(by='need', ascending=False).iloc[0]
                    receiver_id = receiver_row.name
                    receiver_need = receiver_row['need']

                    send_amount = min(sender_capacity, receiver_need)
            
                    current_inventory[sender_id] -= send_amount
                    current_inventory[receiver_id] += send_amount
                    
                    df.at[sender_id, 'max_transship'] -= send_amount
                    df.at[receiver_id, 'need'] -= send_amount
                
                    # log record
                    transfers.append((sender_id, receiver_id, send_amount))
                    
                    weekly_df = pd.concat([weekly_df, pd.DataFrame([{
                        "week": week,
                        "day": current_day,
                        "shipper": sender_id,
                        "receiver": receiver_id,
                        "transshipment_amount": send_amount
        
                }])], ignore_index=True)
                    
                    #print(f"day: {current_day}, current_inventory: {current_inventory}")
                    #print(f"transfers: {transfers}")
                    
                
            elif max_n_amount <= max_t_amount and (max_n_amount != 0):
                
                df = pd.DataFrame.from_dict(current_inventory, orient='index', columns=['inventory'])
                df['max_transship'] = df.index.map(max_transshipments).fillna(0).astype(int)
                df['need'] = df.index.map(max_needs).fillna(0).astype(int)
                transfers = []
                while df['max_transship'].sum() > 0 and df['need'].sum() > 0:
                    # Gönderici: max_transship > 0 and highest
                    sender_row = df[df['max_transship'] > 0].sort_values(by='max_transship', ascending=False).iloc[0]
                    sender_id = sender_row.name
                    sender_capacity = sender_row['max_transship']
                
                    # Alıcı: need > 0 ve highest
                    receiver_row = df[df['need'] > 0].sort_values(by='need', ascending=False).iloc[0]
                    receiver_id = receiver_row.name
                    receiver_need = receiver_row['need']
                
                    # transshipment amount
                    send_amount = min(sender_capacity, receiver_need)
                
                    current_inventory[sender_id] -= send_amount
                    current_inventory[receiver_id] += send_amount
                    
                    df.at[sender_id, 'max_transship'] -= send_amount
                    df.at[receiver_id, 'need'] -= send_amount
                
                    # log record
                    transfers.append((sender_id, receiver_id, send_amount))
                    
                    weekly_df = pd.concat([weekly_df, pd.DataFrame([{
                        "week": week,
                        "day": current_day,
                        "shipper": sender_id,
                        "receiver": receiver_id,
                        "transshipment_amount": send_amount
        
                }])], ignore_index=True)
                    
                    #print(f"day: {current_day}, current_inventory: {current_inventory}")
                    #print(f"transfers: {transfers}")
                    ay + 1
        
    if current_day == 6:
        for i in current_inventory.keys():
                
            mean,stdev=daily_demands[i][current_day]
            number=int(np.random.normal(mean,stdev))
            actual_sales = min(number, current_inventory[i])
            lost_sales = max(0, number - current_inventory[i])
            current_inventory[i]=current_inventory[i]-actual_sales
            new_row = pd.DataFrame({'Store': [i], 'Day': [current_day], 'Sales': actual_sales,'Lost Sales':lost_sales,'Demand':number,'Inventory':current_inventory[i]})
            sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)
    #print(sales_dff)
    weekly_df["transshipment_amount"] = pd.to_numeric(weekly_df["transshipment_amount"], errors="coerce")
    sales_dff["Inventory"] = pd.to_numeric(sales_dff["Inventory"], errors="coerce")
    sales_dff["Lost Sales"]=pd.to_numeric(sales_dff["Lost Sales"], errors="coerce")
    weekly_cost = int(costs["transshipment"])*weekly_df["transshipment_amount"].sum()+int(costs["stockout"])*sales_dff["Lost Sales"].sum()+int(costs["holding"])*sales_dff["Inventory"].sum()
    weekly_df["week_cost"] = weekly_cost
    
    
    results = pd.concat([results, weekly_df], ignore_index=True)
    week = week + 1                    

In [19]:
results.tail()

,week,day,shipper,receiver,transshipment_amount,week_cost
23906,2000,0,5,1,6,4208
23907,2000,0,8,3,5,4208
23908,2000,0,7,6,3,4208
23909,2000,0,5,2,2,4208
23910,2000,0,7,9,1,4208


In [20]:
unique_week_costs = results.drop_duplicates(subset="week")[["week_cost"]]
mean_cost = unique_week_costs["week_cost"].mean()
print(mean_cost)

5025.2065


In [21]:
results["transshipment_amount"].sum()

895282